# Match vz to rs through vz ids

In [1]:
from elasticsearch import Elasticsearch
from exploretools import Explorer
es = Elasticsearch()
ex = Explorer(es)

In [2]:
docs_vz = {'index': 'verejnezakazky_2018-02-13', 
           'doc_type': 'verejnazakazka'}
docs_rs = {'index': 'hlidacsmluv_2018-02-18', 
           'doc_type': 'smlouva'}

In [4]:
ecz = ex.unique_exact(docs_vz, 'evidencniCisloZakazky')
vz_id = ex.unique_exact(docs_vz, 'id')

Went through 185790 entries.
Found 180171 unique entries.
Went through 185790 entries.
Found 185790 unique entries.


In [35]:
def msearch_body(field, vals, limit = None):
    body = []
    for val in vals:
        body.append({})
        body.append({'query': {'term': {field: val}}})
        if limit is not None and len(body)//2 > limit:
            break
    return body

In [39]:
body = msearch_body('prilohy.fileMetadata.value.keyword', ecz)
responses = es.msearch(**docs_vz, body=body)['responses']
print("is there a non-empty response?")
sum([len(response['hits']['hits']) > 0 for response in responses])

is there a non-empty response?


0

In [43]:
body = msearch_body('prilohy.fileMetadata.value.keyword', vz_id)
responses = es.msearch(**docs_vz, body=body)['responses']
print("is there a non-empty response?")
sum([len(response['hits']['hits']) > 0 for response in responses])

GET http://localhost:9200/verejnezakazky_2018-02-13/verejnazakazka/_msearch [status:N/A request:10.074s]
Traceback (most recent call last):
  File "c:\anaconda3\envs\hlidac\lib\site-packages\urllib3\connectionpool.py", line 387, in _make_request
    six.raise_from(e, None)
  File "<string>", line 2, in raise_from
  File "c:\anaconda3\envs\hlidac\lib\site-packages\urllib3\connectionpool.py", line 383, in _make_request
    httplib_response = conn.getresponse()
  File "c:\anaconda3\envs\hlidac\lib\http\client.py", line 1331, in getresponse
    response.begin()
  File "c:\anaconda3\envs\hlidac\lib\http\client.py", line 297, in begin
    version, status, reason = self._read_status()
  File "c:\anaconda3\envs\hlidac\lib\http\client.py", line 258, in _read_status
    line = str(self.fp.readline(_MAXLINE + 1), "iso-8859-1")
  File "c:\anaconda3\envs\hlidac\lib\socket.py", line 586, in readinto
    return self._sock.recv_into(b)
socket.timeout: timed out

During handling of the above exception,

ConnectionTimeout: ConnectionTimeout caused by - ReadTimeoutError(HTTPConnectionPool(host='localhost', port=9200): Read timed out. (read timeout=10))

In [59]:
body = {'query': {'term': {'prilohy.fileMetadata.value.keyword': vz_id}}}
es.search(**docs_rs, body=body)['hits']['hits'][0]

IndexError: list index out of range

In [67]:
body = {'query': {'match': {'prilohy.fileMetadata.value': vz_id}}}
es.search(**docs_rs, body=body)['hits']['hits'][0]

IndexError: list index out of range

In [69]:
body = {'query': {'match': {'prilohy.plainTextContent': vz_id}}}
es.search(**docs_rs, body=body)['hits']['hits'][0]

IndexError: list index out of range

In [87]:
body = [{},
        {'query': {'match': {'evidencniCisloZakazky': '496393'}}},
        {},
        {'query': {'match': {'evidencniCisloZakazky': '644581'}}}]
es.msearch(**docs_vz, body=body)

{'responses': [{'_shards': {'failed': 0,
    'skipped': 0,
    'successful': 5,
    'total': 5},
   'hits': {'hits': [{'_id': '8700CC9E1FF5D48FD7F29DC100B35B85',
      '_index': 'verejnezakazky_2018-02-13',
      '_score': 10.111883,
      '_source': {'cPV': ['16000000-5'],
       'dataset': 'VVZ-2006',
       'datumUverejneni': '2014-09-09T00:00:00',
       'displayId': 0,
       'dodavatele': [],
       'dokumenty': [],
       'evidencniCisloZakazky': '496393',
       'formulare': [{'cislo': '7452010096393',
         'druh': 'F52',
         'onlyOnProfile': False,
         'uRL': '',
         'zverejnen': '2014-09-09T00:00:00'}],
       'id': '8700CC9E1FF5D48FD7F29DC100B35B85',
       'kriteria': [],
       'lastUpdated': '0001-01-01T00:00:00',
       'posledniZmena': '2014-09-09T00:00:00',
       'zadavatel': {'iCO': '62698044', 'jmeno': 'Pavel Francl'}},
      '_type': 'verejnazakazka'}],
    'max_score': 10.111883,
    'total': 1},
   'status': 200,
   'timed_out': False,
   'took